In [1]:
import os

In [2]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    local_data_file2: Path
    unzip_dir: Path

In [12]:
from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            local_data_file2=config.local_data_file2,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [14]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        with zipfile.ZipFile(self.config.local_data_file2, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-07 00:21:59,838: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-07 00:21:59,863: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-07 00:21:59,871: INFO: common: created directory at: artifacts]
[2025-01-07 00:21:59,872: INFO: common: created directory at: artifacts/data_ingestion]


[2025-01-07 00:22:04,313: INFO: 141502029: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 13460480
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "59fe2871e43b2e87cee6ba7a932afc3a8014dee6ec0576da1a10a2c4e5fb4e7f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 37C9:173BD:8734D:8E680:677C73A8
Accept-Ranges: bytes
Date: Tue, 07 Jan 2025 00:22:03 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830049-SIN
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1736209321.926076,VS0,VE2392
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 2e764a138326421034445601ad46e7a82c42f809
Expires: Tue, 07 Jan 2025 00:27:03 GMT
Source-Age: 2

]
